<a href="https://colab.research.google.com/github/ranieri-unimi/faces-malchiodi-2022/blob/main/faces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### run once

In [1]:
!pip install pyspark
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 49 kB/s 
     |████████████████████████████████| 199 kB 59.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=f1ffb4f3ead9f2d72a02ae255116abcd09a7c3c6da2786c92f4d6371f2a02776
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
os.environ["KAGGLE_USERNAME"] = 'ranieriunimi'
os.environ["KAGGLE_KEY"] = str(hex(232307088475198570779809482024044346960))[2:]

In [3]:
ref = 'bwandowando/ukraine-russian-crisis-twitter-dataset-1-2-m-rows'
!mkdir datasets
!kaggle datasets download $ref --unzip -p ./datasets

100% 9.82G/9.85G [00:47<00:00, 233MB/s]
100% 9.85G/9.85G [00:47<00:00, 222MB/s]


## ukraine

In [4]:
SAMPLE_SIZE = 10*1000

In [5]:
import numpy as np
import pandas as pd
import csv
import re
import string
import random

In [6]:
import pyspark
import findspark

In [7]:
# load dataset 
filename = r"./datasets/UkraineCombinedTweetsDeduped20220227-131611.csv.gzip"
pd.set_option("display.max_columns", None)
df = pd.read_csv(filename, compression='gzip', index_col=0, encoding='utf-8', quoting=csv.QUOTE_ALL)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## data cleaning

In [8]:
df = df[df.language == 'en']

In [9]:
df = df.text.tolist()

In [10]:
df = random.sample(df, SAMPLE_SIZE) if SAMPLE_SIZE else df

In [11]:
def preprocess_tweet_text(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
    tweet = tweet.translate(str.maketrans(string.punctuation+'…’”“‘', ' '*(len(string.punctuation)+5)))  # puntctuation to spaces
    tweet = ' '.join(tweet.strip().split())
    return tweet

## hadoop instance

In [12]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [13]:
rdd = sc.parallelize(enumerate(df))

In [14]:
rdd.take(7)

[(0,
  '"DO NOT DESPAIR... THE MISERY THAT IS NOW UPON US IS WORTH THE PASSING OF GREED... "\n#WWIII #worldwar3 #china #RussiaUkraine https://t.co/naJHVR0NOl'),
 (1, 'Atrillery attack from #Russia on #Ukraine https://t.co/svtRxjz8UN'),
 (2, 'Cut #Russia off from #Swift now'),
 (3, '#BREAKING: Mayor of Ukraine capital Kyiv orders curfew. (TASS)'),
 (4,
  'On #Ukraine, PM @ImranKhanPTI has shared his belief that disputes should be resolved through dialogue and diplomacy - conflict is not in anyone’s interest. In terms of evacuations of the Pakistani community, @PakinUkraine is working round the clock to ensure safe evacuation.'),
 (5,
  '#Putin has brainwashed the Russian military to attack Ukraine. \n\nJust like how #Trump brainwashed the #MAGA cult to attack the US Capitol.'),
 (6,
  'Reports coming out of Ukraine of a soldier being nicknamed as the ‘Ukrainian Reaper’ who has killed over 20 Russian soldiers on his own.\n\nFollow us @UkraineRussiaa \n.\n#RussiaUkraine #UkraineUnderAttac

In [15]:
def shiftkv(x) : return (x[-1], x[0])

In [16]:
dataPipe = (rdd
            .mapValues(preprocess_tweet_text)
            .map(shiftkv)
            .reduceByKey(min)
            .map(shiftkv)
)

In [17]:
dataPipe.take(7)

[(0,
  'do not despair the misery that is now upon us is worth the passing of greed wwiii worldwar3 china russiaukraine'),
 (2, 'cut russia off from swift now'),
 (3, 'breaking mayor of ukraine capital kyiv orders curfew tass'),
 (6,
  'reports coming out of ukraine of a soldier being nicknamed as the ukrainian reaper who has killed over 20 russian soldiers on his own follow us ukrainerussiaa russiaukraine ukraineunderattack ukrainians russiaukrainewar ukraine russianinvasion russiainvadedukraine'),
 (8, 'iran s calls for political solution to ukraine crisis ukrainerussia'),
 (10,
  'ukraine the man whose car was rammed and crushed by alleged russian strela has been rescued kyiv oblast'),
 (11, 'find it so true man ukrainerussiawar unsc russiaukrainewar zelensky')]

In [18]:
dataPipe.count()

6767

### foos

In [19]:
!pip install crc16
import crc16
import binascii

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for crc16: filename=crc16-0.1.1-cp37-cp37m-linux_x86_64.whl size=22884 sha256=c3a22f7db6dd2dc26496a42ca87259e7392cefa0d6932d59c6dff1daf29cc347
  Stored in directory: /root/.cache/pip/wheels/2b/28/c3/e6303cf55de8a79c8c4b1528bd4b5f577d85debdbe916c0e2c
Successfully built crc16


### shingles

In [40]:
SH_LEN = 5

In [41]:
shigles = dataPipe.flatMap(lambda x : [(x[0], x[-1][i:i+SH_LEN]) for i in range(len(x[-1])-SH_LEN)]).distinct()

In [42]:
shigles.take(7)

[(0, 'do no'),
 (0, 'o not'),
 (0, ' not '),
 (0, 'ot de'),
 (0, 't des'),
 (0, ' desp'),
 (0, 'despa')]

In [43]:
def hashval(x):
  bval = bytes(x, 'utf-8')
  hval = binascii.crc32(bval) # 4 bytes
  return hval

In [44]:
charactMx = shigles.mapValues(hashval).distinct()

In [45]:
charactMx.take(7)

[(0, 2318834904),
 (0, 4021045262),
 (0, 1025072606),
 (0, 4095394484),
 (0, 3847610256),
 (0, 4060343064),
 (0, 637951864)]

### MinHashing

In [68]:
def RN(stop, start = 1) : return random.randrange(start,stop)
def RB(B = 4) : return RN(2**(8*B-1),0)*2+1
def PF(x, scale, shift, base) : return (scale * x + shift) % base

In [69]:
import random
n_permfoo, b_bands = 120, 12

(1 / b_bands)**(1 / (n_permfoo//b_bands)) # threshold

0.7799771419043033

In [70]:
params = [ {'shift':RB(), 'scale':RB(), 'base':2**(8*4)} for _ in range(n_permfoo) ]
params

[{'base': 4294967296, 'scale': 2013516383, 'shift': 2130726695},
 {'base': 4294967296, 'scale': 1544659765, 'shift': 1887084175},
 {'base': 4294967296, 'scale': 2017383223, 'shift': 1327856977},
 {'base': 4294967296, 'scale': 998478385, 'shift': 441888351},
 {'base': 4294967296, 'scale': 2946498111, 'shift': 494915693},
 {'base': 4294967296, 'scale': 1556139163, 'shift': 2874047961},
 {'base': 4294967296, 'scale': 1082911121, 'shift': 3832844083},
 {'base': 4294967296, 'scale': 3657518145, 'shift': 30080683},
 {'base': 4294967296, 'scale': 3388874721, 'shift': 3537795959},
 {'base': 4294967296, 'scale': 2170844501, 'shift': 715464809},
 {'base': 4294967296, 'scale': 851762691, 'shift': 1712500825},
 {'base': 4294967296, 'scale': 3979007389, 'shift': 358842887},
 {'base': 4294967296, 'scale': 342419601, 'shift': 1555118181},
 {'base': 4294967296, 'scale': 2978732355, 'shift': 2993760945},
 {'base': 4294967296, 'scale': 2462155481, 'shift': 1927158107},
 {'base': 4294967296, 'scale': 179

In [71]:
def gen_perm(x):
  doc, shingle = x
  return [ ((doc, h), (shingle, h) ) for h in range(n_permfoo)]

In [72]:
minHashSignMx = (charactMx
                 .flatMap(gen_perm)
                 .reduceByKey(lambda a, b : a if PF(a[0], **params[a[-1]]) < PF(b[0], **params[a[-1]]) else b)
                 .mapValues(lambda v : v[0])
)
# (doc, hash) , shingle

In [73]:
minHashSignMx.take(7)

[((0, 0), 531944130),
 ((0, 2), 3847610256),
 ((0, 4), 1400433706),
 ((0, 6), 3653533309),
 ((0, 8), 3585268625),
 ((0, 10), 4175520368),
 ((0, 12), 567660810)]

### LSH

In [74]:
scale, shift = RN(b_bands), RN(b_bands)
bandmap = (lambda x : PF(x, scale, shift, b_bands))

In [75]:
def band_expand(x):
  (doc, h), v = x
  return ((doc, bandmap(h)), (h, v))

def band_reduct(x):
  (doc, band), a = x
  return ((hash(tuple(a)), band), doc)

In [76]:
bandMx = (minHashSignMx              # (doc, hashperm) , valshin
          .map(band_expand)             # (doc, band) , (hashperm , valshin)
          .groupByKey().mapValues(list)    # (doc, band) , [ (hashperm , valshin), (hashperm , valshin) ,(hashperm , valshin), ...]
          .mapValues(lambda a : sorted(a, key=lambda x: x[0]))   # (doc, band) , [ (hashperm , valshin), (hashperm , valshin) ,(hashperm , valshin), ...] but sorted
          .mapValues(lambda a : [v for _, v in a] )      # (doc, band) , [valshin, valshin, valshin]
)

In [77]:
bandMx.take(7)

[((0, 6),
  [901026131,
   2959325322,
   531944130,
   2454390766,
   2891949024,
   1988200157,
   518546981,
   3847610256,
   551827451,
   1720809072]),
 ((0, 4),
  [305718398,
   2870458300,
   76905862,
   3473581423,
   810485567,
   4293022898,
   1347333239,
   4282597887,
   4095394484,
   2454390766]),
 ((0, 2),
  [3708505255,
   901026131,
   2983904561,
   3277007011,
   3604591150,
   2983904561,
   1571788597,
   2846681743,
   231363539,
   3296795597]),
 ((0, 0),
  [147679802,
   1720809072,
   2398395017,
   2931046475,
   4282597887,
   942146670,
   1843785225,
   637951864,
   901026131,
   2982624464]),
 ((0, 10),
  [1397413727,
   3585268625,
   2982624464,
   2738985294,
   1026278435,
   1626325540,
   2454390766,
   2132423132,
   2959325322,
   942146670]),
 ((0, 8),
  [687136356,
   2685941870,
   3051758138,
   3653533309,
   1571788597,
   625309798,
   4293022898,
   4095394484,
   2891949024,
   3949197236]),
 ((2, 6),
  [3187861507,
   3117458648,
   2

In [78]:
bandBuckets = (bandMx
    .map(band_reduct)                           # (HHH_valshin, band), doc
    .groupByKey().mapValues(list)           # (HHH_valshin, band), [doc, doc, doc]
)

In [79]:
lst = bandBuckets.filter(lambda x : len(x[1])>1).collect()
lst[:7]

In [83]:
adjlst = {tuple(sorted([v[i],v[j]])) for k, v in lst for i in range(len(v)-1) for j in range(i+1, len(v))}

In [91]:
import networkx as nx
G = nx.Graph()
G.add_edges_from(adjlst)
canditates = [cc for cc in nx.connected_components(G) if len(cc) > 3]

In [92]:
for v in canditates:
  for i in v:
    print(df[i])
    print('---------------')
  print('___________________________________________________________________________________________')

“When you attack us , You will see our faces , not our backs “
~ #Ukraine president ✨ https://t.co/t3FuzrOJlQ
---------------
“When you attack us, You will see our faces, not our backs”

~ Ukraine president ✊❤️ 🔥
#UkraineRussiaWar #Russia #Ukraine 
#UkraineRussia #HelpUkraine https://t.co/LOFB2KrhYr
---------------
“When you attack us, You will see our faces, not our backs”
~ Ukraine President Zelensky  🇺🇦 

#UkraineRussiaWar #Russia #Ukraine #UkraineUnderAttack #UkraineRussiaConflict #UkraineRussiaCrisis https://t.co/BSNsq8UvH7
---------------
“When you attack us, You will see our faces, not our backs” #ukraine #Zelensky #UkraineUnderAttack 🌻 🇺🇦 https://t.co/VJUDrgw9gu
---------------
“When you attack us, You will see our faces, not our backs”

~ Ukraine president ✊❤️ 🔥
#UkraineRussiaWar #Russia #Ukraine 
https://t.co/HlYzQwu4rf https://t.co/Yh2elSTMgw
---------------
“When you attack us, You will see our faces, not our backs”~ Ukraine president ✊❤️ 
BE STRONG, BE COURAGEOUS 💪
#Ukrain